In [1]:
import pandas as pd
import numpy as np

In [2]:
df_teamstats = pd.read_csv("data/AmericanFootball98.csv")
df_sb = pd.read_csv("data/superbowl.csv")

In [3]:
df_teamstats.drop(df_teamstats.columns[33:], axis=1, inplace=True)
df_sb.drop(df_sb.columns[6:], axis=1, inplace=True)
df_sb.drop(0, inplace=True)

#changes date to year
df_sb['Year'] = pd.to_datetime(df_sb['Date']).dt.year
df_sb.drop(labels = 'Date', axis=1, inplace=True)

#changes sb roman number to just int
df_sb['SB'] = df_sb['SB'].str.replace(r'[a-zA-Z]', '', regex=True)
df_sb['SB'] = df_sb['SB'].str.replace(r'\(|\)', '', regex=True)

#moves year
column_to_move = df_sb.pop("Year")
df_sb.insert(0, "Year", column_to_move)

In [4]:
#Extracting the team names and years from df_teamstats
df_teamstats['Team'] = df_teamstats['team_code'].str.extract(r'([A-Za-z]{3})')
df_teamstats['Year'] = df_teamstats['team_code'].str.extract(r'(\d{4})')

#reorder columns
column_to_move = df_teamstats.pop("Year")
df_teamstats.insert(0, "Year", column_to_move)
column_to_move = df_teamstats.pop("Team")
df_teamstats.insert(1, "Team", column_to_move)

#Year str to int
df_teamstats['Year'] = df_teamstats['Year'].astype(int)

#delete team_code
df_teamstats = df_teamstats.drop('team_code', axis=1)
df_teamstats

,Year,Team,wins,losses,PF,yards,plays,yards/play,TO,Fumbles Lost,...,pen yards,pen 1st downs,number drives,score percentage,turnover percentage,avg start,avg time per drive,avg plays per drive,avg yards per drive,avg points per drive
0,2019,nwe,12,4,420,5664,1095,5.2,15,6,...,828,31,185,36.8,7.6,Own 32.5,2:40,5.97,29.9,1.99
1,2019,buf,10,6,314,5283,1018,5.2,19,7,...,927,32,183,30.6,10.4,Own 28.9,2:34,5.62,28.6,1.66
2,2019,nyj,7,9,276,4368,956,4.6,25,9,...,1105,30,183,23.0,11.5,Own 26.4,2:28,5.23,23.1,1.21
3,2019,mia,5,11,306,4960,1022,4.9,26,8,...,769,41,180,30.6,13.3,Own 27.6,2:33,5.78,27.1,1.63
4,2019,rav,14,2,531,6521,1064,6.1,15,7,...,867,27,163,52.1,8.6,Own 28.9,3:22,6.61,39.3,2.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,1999,ram,13,3,526,6412,994,6.5,31,16,...,889,26,188,39.9,15.4,Own 31.5,2:42,5.44,34.1,2.37
665,1999,car,8,8,421,5686,982,5.8,34,19,...,857,21,192,38.0,15.1,Own 32.7,2:27,5.26,29.6,2.14
666,1999,atl,5,11,285,4542,931,4.9,35,16,...,968,26,184,25.0,17.4,Own 30.2,2:30,5.17,24.6,1.42
667,1999,sfo,4,12,295,5380,1012,5.3,32,13,...,1045,26,182,26.9,15.9,Own 28.5,2:40,5.69,29.5,1.42


In [5]:
#Dictionaries
sb_dict = {
    "New England Patriots":"Patriots", #won 4 from 2005-2019
    "Miami Dolphins":"Dolphins",
    "Buffalo Bills":"Bills",
    "New York Jets":"Jets",
    "Cincinnati Bengals":"Bengals",
    "Pittsburgh Steelers":"Steelers", #won 2 from 2005-2019
    "Baltimore Ravens":"Ravens", #won 1 from 2005-2019
    "Cleveland Browns": "Browns", #team has not been to the SB yet
    "Baltimore Colts":"Colts",
    "Indianapolis Colts":"Colts", #2nd Colts Team, #won 1 from 2005-2019
    "Jacksonville Jaguars":"Jaguars", #team has not been to the SB yet
    "Tennessee Titans":"Titans", 
    "Houston Texans":"Texans", #team has not been to the SB yet
    "Denver Broncos":"Broncos", #won 1 from 2005-2019
    "Kansas City Chiefs":"Chiefs",
    "San Diego Chargers":"Chargers", #omitted LA Chargers, no SB appearance
    "Oakland Raiders":"Raiders", #omitted LV Raiders, no SB appearance
    "Los Angeles Raiders":"Raiders", #2nd Raiders Team
    "New York Giants":"Giants", #won 2 from 2005-2019
    "Washington Redskins":"Redskins", #omitted Commanders, no SB appearance
    "Dallas Cowboys":"Cowboys",
    "Philadelphia Eagles":"Eagles", #won 1 from 2005-2019
    "Chicago Bears":"Bears",
    "Minnesota Vikings":"Vikings",
    "Detroit Lions":"Lions", #team has not been to SB yet
    "Green Bay Packers":"Packers", #won 1 from 2005-2019
    "Tampa Bay Buccaneers":"Buccaneers",
    "Carolina Panthers":"Panthers",
    "Atlanta Falcons":"Falcons",
    "New Orleans Saints":"Saints", #won 1 from 2005-2019
    "Seattle Seahawks":"Seahawks", #won 1 from 2005-2019
    "St. Louis Rams":"Rams",
    "Los Angeles Rams":"Rams", #2nd Rams Team
    "Arizona Cardinals":"Cardinals",
    "San Francisco 49ers":"49ers"
}

team_dict = {
    "nwe": "Patriots",
    "mia":"Dolphins",
    "buf":"Bills",
    "nyj":"Jets",
    "cin":"Bengals",
    "pit":"Steelers",
    "rav":"Ravens",
    "cle":"Browns",
    "clt":"Colts",
    "jax":"Jaguars",
    "oti":"Titans",
    "htx":"Texans",
    "den":"Broncos",
    "kan":"Chiefs",
    "sdg":"Chargers",
    "rai":"Raiders",
    "nyg":"Giants",
    "was":"Redskins",
    "dal":"Cowboys",
    "phi":"Eagles",
    "chi":"Bears",
    "min":"Vikings",
    "det":"Lions",
    "gnb":"Packers",
    "tam":"Buccaneers",
    "car":"Panthers",
    "atl":"Falcons",
    "nor":"Saints",
    "sea":"Seahawks",
    "ram":"Rams",
    "crd":"Cardinals",
    "sfo":"49ers"
}

In [6]:
#changes team names to mascots
df_sb['Winner'] = df_sb['Winner'].replace(sb_dict)
df_sb['Loser'] = df_sb['Loser'].replace(sb_dict)

df_teamstats['Team'] = df_teamstats['Team'].replace(team_dict)

df_sb.head(10)

,Year,SB,Winner,Winner Pts,Loser,Loser Pts
1,2019,53,Patriots,13,Rams,3
2,2018,52,Eagles,41,Patriots,33
3,2017,51,Patriots,34,Falcons,28
4,2016,50,Broncos,24,Panthers,10
5,2015,49,Patriots,28,Seahawks,24
6,2014,48,Seahawks,43,Broncos,8
7,2013,47,Ravens,34,49ers,31
8,2012,46,Giants,21,Patriots,17
9,2011,45,Packers,31,Steelers,25
10,2010,44,Saints,31,Colts,17


In [7]:
df_teamstats

,Year,Team,wins,losses,PF,yards,plays,yards/play,TO,Fumbles Lost,...,pen yards,pen 1st downs,number drives,score percentage,turnover percentage,avg start,avg time per drive,avg plays per drive,avg yards per drive,avg points per drive
0,2019,Patriots,12,4,420,5664,1095,5.2,15,6,...,828,31,185,36.8,7.6,Own 32.5,2:40,5.97,29.9,1.99
1,2019,Bills,10,6,314,5283,1018,5.2,19,7,...,927,32,183,30.6,10.4,Own 28.9,2:34,5.62,28.6,1.66
2,2019,Jets,7,9,276,4368,956,4.6,25,9,...,1105,30,183,23.0,11.5,Own 26.4,2:28,5.23,23.1,1.21
3,2019,Dolphins,5,11,306,4960,1022,4.9,26,8,...,769,41,180,30.6,13.3,Own 27.6,2:33,5.78,27.1,1.63
4,2019,Ravens,14,2,531,6521,1064,6.1,15,7,...,867,27,163,52.1,8.6,Own 28.9,3:22,6.61,39.3,2.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,1999,Rams,13,3,526,6412,994,6.5,31,16,...,889,26,188,39.9,15.4,Own 31.5,2:42,5.44,34.1,2.37
665,1999,Panthers,8,8,421,5686,982,5.8,34,19,...,857,21,192,38.0,15.1,Own 32.7,2:27,5.26,29.6,2.14
666,1999,Falcons,5,11,285,4542,931,4.9,35,16,...,968,26,184,25.0,17.4,Own 30.2,2:30,5.17,24.6,1.42
667,1999,49ers,4,12,295,5380,1012,5.3,32,13,...,1045,26,182,26.9,15.9,Own 28.5,2:40,5.69,29.5,1.42


In [8]:
# drop all rows from 2004 and older
df_sb = df_sb[df_sb['Year'] > 2004]
df_teamstats = df_teamstats[df_teamstats['Year'] > 2004]

#merging df together
df_merged = pd.merge(df_teamstats, df_sb, on='Year', how='left')

df_merged

,Year,Team,wins,losses,PF,yards,plays,yards/play,TO,Fumbles Lost,...,avg start,avg time per drive,avg plays per drive,avg yards per drive,avg points per drive,SB,Winner,Winner Pts,Loser,Loser Pts
0,2019,Patriots,12,4,420,5664,1095,5.2,15,6,...,Own 32.5,2:40,5.97,29.9,1.99,53,Patriots,13,Rams,3
1,2019,Bills,10,6,314,5283,1018,5.2,19,7,...,Own 28.9,2:34,5.62,28.6,1.66,53,Patriots,13,Rams,3
2,2019,Jets,7,9,276,4368,956,4.6,25,9,...,Own 26.4,2:28,5.23,23.1,1.21,53,Patriots,13,Rams,3
3,2019,Dolphins,5,11,306,4960,1022,4.9,26,8,...,Own 27.6,2:33,5.78,27.1,1.63,53,Patriots,13,Rams,3
4,2019,Ravens,14,2,531,6521,1064,6.1,15,7,...,Own 28.9,3:22,6.61,39.3,2.96,53,Patriots,13,Rams,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,2005,Saints,3,13,235,5031,1017,4.9,43,19,...,Own 28.5,2:41,5.77,27.6,1.30,39,Patriots,24,Eagles,21
476,2005,Seahawks,13,3,452,5915,1020,5.8,17,7,...,Own 31.6,2:35,5.65,31.9,2.34,39,Patriots,24,Eagles,21
477,2005,Rams,6,10,363,5571,1025,5.4,37,13,...,Own 30.6,2:30,5.42,28.5,1.71,39,Patriots,24,Eagles,21
478,2005,Cardinals,5,11,311,5575,1075,5.2,37,16,...,Own 29.5,2:35,5.77,28.7,1.49,39,Patriots,24,Eagles,21


In [9]:
mascottocode = {
    "Patriots": 1,
    "Dolphins": 2,
    "Bills":3,
    "Jets":4,
    "Bengals":5,
    "Steelers":6,
    "Ravens":7,
    "Browns":8,
    "Colts":9,
    "Jaguars":10,
    "Titans":11,
    "Texans":12,
    "Broncos":13,
    "Chiefs":14,
    "Chargers":15,
    "Raiders":16,
    "Giants":17,
    "Redskins":18,
    "Cowboys":19,
    "Eagles":20,
    "Bears":21,
    "Vikings":22,
    "Lions":23,
    "Packers":24,
    "Buccaneers":25,
    "Panthers":26,
    "Falcons":27,
    "Saints":28,
    "Seahawks":29,
    "Rams":30,
    "Cardinals":31,
    "49ers":32
}
codetomascot = {
    1:"Patriots",
    2:"Dolphins",
    3:"Bills",
    4:"Jets",
    5:"Bengals",
    6:"Steelers",
    7:"Ravens",
    8:"Browns",
    9:"Colts",
    10:"Jaguars",
    11:"Titans",
    12:"Texans",
    13:"Broncos",
    14:"Chiefs",
    15:"Chargers",
    16:"Raiders",
    17:"Giants",
    18:"Redskins",
    19:"Cowboys",
    20:"Eagles",
    21:"Bears",
    22:"Vikings",
    23:"Lions",
    24:"Packers",
    25:"Buccaneers",
    26:"Panthers",
    27:"Falcons",
    28:"Saints",
    29:"Seahawks",
    30:"Rams",
    31:"Cardinals",
    32:"49ers"
}

In [10]:
df_merged['Team'] = df_merged['Team'].replace(mascottocode)
df_merged['Winner'] = df_merged['Winner'].replace(mascottocode)
df_merged['Loser'] = df_merged['Loser'].replace(mascottocode)
df_merged

,Year,Team,wins,losses,PF,yards,plays,yards/play,TO,Fumbles Lost,...,avg start,avg time per drive,avg plays per drive,avg yards per drive,avg points per drive,SB,Winner,Winner Pts,Loser,Loser Pts
0,2019,1,12,4,420,5664,1095,5.2,15,6,...,Own 32.5,2:40,5.97,29.9,1.99,53,1,13,30,3
1,2019,3,10,6,314,5283,1018,5.2,19,7,...,Own 28.9,2:34,5.62,28.6,1.66,53,1,13,30,3
2,2019,4,7,9,276,4368,956,4.6,25,9,...,Own 26.4,2:28,5.23,23.1,1.21,53,1,13,30,3
3,2019,2,5,11,306,4960,1022,4.9,26,8,...,Own 27.6,2:33,5.78,27.1,1.63,53,1,13,30,3
4,2019,7,14,2,531,6521,1064,6.1,15,7,...,Own 28.9,3:22,6.61,39.3,2.96,53,1,13,30,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,2005,28,3,13,235,5031,1017,4.9,43,19,...,Own 28.5,2:41,5.77,27.6,1.30,39,1,24,20,21
476,2005,29,13,3,452,5915,1020,5.8,17,7,...,Own 31.6,2:35,5.65,31.9,2.34,39,1,24,20,21
477,2005,30,6,10,363,5571,1025,5.4,37,13,...,Own 30.6,2:30,5.42,28.5,1.71,39,1,24,20,21
478,2005,31,5,11,311,5575,1075,5.2,37,16,...,Own 29.5,2:35,5.77,28.7,1.49,39,1,24,20,21
